# About 
Here, I build a Text-to-SQL query engine to query a database (built from `00-book-database.ipynb`) 
- The LLM model I used is Meta's `llama2 7B` model (Ollma ID `78e26419b446`).


# Settings

### Packages

In [1]:
import os
import pandas as pd
from tqdm import tqdm

# sql related
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists

# llama_index
from llama_index.core import SQLDatabase
from llama_index.core.indices.vector_store.base import ServiceContext
from llama_index.core.indices.struct_store.sql_query import NLSQLTableQueryEngine


# llm model
from langchain.llms import Ollama


### Variables

In [2]:
#----------------#
# variables that require changes
#----------------#
llm_model_id ="llama2"


# databse name
db_name =  "pg_catalog.db"

# sql table name 
sql_table_name = 'catalog_table'



### Directories

In [3]:
main_Dir = "../"

#----------------#
# data dir
#----------------#
data_Dir = os.path.join(main_Dir,"data")
sql_data_Dir=os.path.join(data_Dir,"sql")


#----------------#
# make dirs
#----------------#
for f in [data_Dir, sql_data_Dir]:
    os.makedirs(f, exist_ok=True)

# Connect to a database 

In [4]:
# database path
db_path= os.path.join(sql_data_Dir,db_name)
print(f"db path:{db_path}")

db path:../data/sql/pg_catalog.db


In [5]:
# create a sql engine
sql_url = f'sqlite:///{db_path}'

#Construct a SQLDatabase Index
catalog_db = SQLDatabase.from_uri(sql_url,
                         include_tables=[sql_table_name])

### Setup a model 

In [6]:
# model
llm = Ollama(model=llm_model_id)

# context
servicecontext = ServiceContext.from_defaults(llm=llm,
                                              embed_model="local",
                                              chunk_size=500)

/var/folders/w9/110177p12dsgs8fkfpx2h51m0000gn/T/ipykernel_59860/99248196.py:5: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  servicecontext = ServiceContext.from_defaults(llm=llm,


In [7]:
# query engine
sql_query_engine = NLSQLTableQueryEngine(sql_database=catalog_db,
                                         tables=[sql_table_name],
                                         service_context=servicecontext)

# Ask Questions
### Select 5 books from a certain category

In [9]:
# the type of books
selection_keyword ="Humor"

# Specify where to query
selection_column = "Bookshelves"

# Form the question
selection_question= f'Using the provided table "{sql_table_name}", return the top 5 "Book" along with their "ID" from the "{selection_keyword}" "{selection_column}".'
selection_question

'Using the provided table "catalog_table", return the top 5 "Book" along with their "ID" from the "Humor" "Bookshelves".'

In [10]:
selection_query= sql_query_engine.query(selection_question)
selection_query.metadata["result"]

/Users/j/miniconda3/envs/library/lib/python3.11/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


[('Wit and Wisdom of Don Quixote', 24754),
 ('Winsome Winnie and other New Nonsense Novels', 20633),
 ('William Tell Told Again', 7298),
 ('Vice Versa; or, A Lesson to Fathers', 26853),
 ('Uneasy Money', 6684)]

### Select 5 books from another category

In [17]:
# the type of books
selection_keyword ="Science Fiction"

# Specify where to query
selection_column = "Bookshelves"

# Form the question
selection_question= f'Using the provided table "{sql_table_name}", return the top 10 "Book" along with their "ID" from the "{selection_keyword}" "{selection_column}".'
selection_question

'Using the provided table "catalog_table", return the top 10 "Book" along with their "ID" from the "Science Fiction" "Bookshelves".'

In [18]:
selection_query= sql_query_engine.query(selection_question)
selection_query.metadata["result"]

[('—And Devious the Line of Duty', 22585),
 ('Zero Hour', 30476),
 ('Zero Data', 29727),
 ("Zeritsky's Law", 51234),
 ('Zen', 29750),
 ('Zehru of Xollar', 26917),
 ('Zarlah the Martian', 13423),
 ('Youth', 31547),
 ("You Don't Make Wine Like the Greeks Did", 31897),
 ('Yesterday House', 50905)]